### Multi Agent-Systems
- A team is a group of agents that work together to achieve a common goal

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENROUTER_API_KEY"] = os.getenv("OPENROUTER_API_KEY")

In [2]:
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    model="mistralai/mistral-small-3.2-24b-instruct:free",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    model_info={
        "family": "mistral",
        "vision": True,
        "function_calling": True,
        "json_output": False,
        "structured_output": False
    }
)

In [3]:
model_client

#### Multi Agent Team Approach - With Trmination Condictions
- We will be using 4 agents to now write the story, each with its own work and mastery

In [4]:
from autogen_agentchat.agents import AssistantAgent

In [5]:
plot_agent = AssistantAgent(
    name="plot_writer",
    model_client=model_client,
    system_message="You create engaging plots for stories. Focus on the Pokemon's journey"
)

In [6]:
character_agent = AssistantAgent(
    name="character_writer",
    model_client=model_client,
    system_message="You develop characters. Describe the pokemon and the villian in detail, including their motivations and backgrounds."
)

In [7]:
ending_agent = AssistantAgent(
    name="ending_writer",
    model_client=model_client,
    system_message="You wrute engaging endings. conclude the story with a twist."
)

In [8]:
review_agent = AssistantAgent(
    name = 'review_writer',
    model_client=model_client,
    system_message="You have to say 'APPROVE' once the story is complete and you like the full flow else provide your feedback."
)

#### Termination Conditions :
- Termination Conditions Is Used In Simple Terms and The Name Suggest To Terminate The Team To Complete The Task as If We Didnot Set **max_turns** Would Run Lots Of Time and May Be More Than Needed That Costs You Only.

In [9]:
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination

termination_condition = MaxMessageTermination(10) | TextMentionTermination('APPROVE')

#### **RoundRobinGroupChat** 
- Is a simple yet effective team configuration where all agents share the same context and take turns responding in a round-robin fashion. Each agent, during its turn, broadcasts its response to all other agents, ensuring that the entire team maintains a consistent context.

In [10]:
from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    participants=[plot_agent, character_agent, ending_agent, review_agent],
    termination_condition=termination_condition
)

In [11]:
team

In [12]:
team.component_type

'team'

In [13]:
from autogen_agentchat.base import TaskResult

# When running inside a script, use a async main function and call it from `asyncio.run(...)`.

await team.reset()  # Reset the team for a new task.

async for message in team.run_stream(task="Write a short poem about the fall season."):
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

id='aa2628e1-2eaa-489e-a1c0-0e18b76a4630' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 8, 4, 23, 6, 18, 491652, tzinfo=datetime.timezone.utc) content='Write a short poem about the fall season.' type='TextMessage'
id='798eb235-f7de-44eb-9250-3969351544fd' source='plot_writer' models_usage=RequestUsage(prompt_tokens=28, completion_tokens=142) metadata={} created_at=datetime.datetime(2025, 8, 4, 23, 6, 24, 650864, tzinfo=datetime.timezone.utc) content="In the realm where autumn's whispers call,\nA dance of leaves, a vivid ball.\nCrimson, gold, the trees take flight,\nPainting skies with fleeting light.\n\nCrisp the air, a chilly breeze,\nRustling through the rustling leaves.\nPumpkins grin, their lanterns glow,\nOn this calm and winding road.\n\nFootsteps crunch on fallen hues,\nThrough the woods, my journey views.\nEvery path, a tale untold,\nIn this season, brave and bold.\n\nSo I walk, with heart alight,\nIn the beauty, pure and bright.\nFor in autumn'

In [14]:
from autogen_agentchat.base import TaskResult

# When running inside a script, use a async main function and call it from `asyncio.run(...)`.

await team.reset()  # Reset the team for a new task.

#### Take Care APPROVE Added To The task ????? 😂
async for message in team.run_stream(task="Write a short poem about the fall season APPROVE."):
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

id='90408721-6854-4c06-a70a-8151913d644c' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 8, 4, 23, 7, 4, 48878, tzinfo=datetime.timezone.utc) content='Write a short poem about the fall season APPROVE.' type='TextMessage'
Stop Reason: Text 'APPROVE' mentioned
